In [1]:
import pandas_datareader.data as web
import requests
import config

In [2]:
company = 'ROBLOX'
ticker = 'RBLX'
api = config.fmp_api_key

In [3]:
#ROBLOX EXAMPLE
roblox_bs = requests.get(f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?apikey={api}').json()
roblox_is = requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{ticker}?apikey={api}').json()
roblox_mc = requests.get(f'https://financialmodelingprep.com/api/v3/market-capitalization/{ticker}?apikey={api}').json()

### Market Equity of Value

In [4]:
mkt_value_of_equity = roblox_mc[0]['marketCap']
print("Market value of equity for %s is %.02f billion" % (company, mkt_value_of_equity))

Market value of equity for ROBLOX is 26441558016.00 billion


In [5]:
def total_book_debt(current_portion_LT_debt, long_term_Debt, notes_payable):
    """
    Calculate the total debt based off balance sheet data, current_portion_LT_debt, long_term_Debt and notes_payable.
    """
    return current_portion_LT_debt + long_term_Debt + notes_payable

# ROBLOX EXAMPLE
cltd = 0
ld = roblox_bs[0]['longTermDebt']
np = 0

total_book_debt = total_book_debt(cltd, ld, np)
print("Total debt for %s is %.02f billion" % (company, total_book_debt))

Total debt for ROBLOX is 1182339000.00 billion


In [6]:
def mv_debt_by_average_maturity(average_maturity, cost_of_debt, total_book_debt, interest_expense):
    """
    Calculate the market value of debt based off financial statement data, cost of debt, and average maturity.
    """
    coupon_rate = interest_expense / total_book_debt
    principal = total_book_debt
    coupon_payment = coupon_rate * principal
    
    return coupon_payment * ((1 - (1 + cost_of_debt)**(-average_maturity))/cost_of_debt) + principal/(1 + cost_of_debt)**average_maturity

# ROBLOX EXAMPLE
interest_expense = 6998000.0
cost_of_debt = 0.159
average_maturity = 5

mkt_value_of_debt = mv_debt_by_average_maturity(average_maturity, cost_of_debt, total_book_debt, interest_expense)
print("Market value of debt for %s is %.02f million" % (company, mkt_value_of_debt))

Market value of debt for ROBLOX is 588326743.72 million


In [7]:
def capital_weights(total_debt, common_stock):
    """
    Summary: Given a firm's capital structure, calculate the weights of each group.
    
    PARA total_capital: The company's total capital.
    PARA type: float
    
    PARA common_stock: The company's common stock outstanding.
    PARA type: float
    
    PARA total_debt: The company's total debt.
    PARA type: float
    
    RTYP weights_dict: A dictionary of all the weights. 
    RTYP weights_dict: dictionary
    
    """      
    # initalize the dictionary
    weights_dict = {}
    
    # calculate the total capital
    total_capital = common_stock + total_debt
    
    # calculate each weight and store it in the dictionary
    weights_dict['common_stock'] = common_stock / total_capital
    weights_dict['total_debt'] = total_debt / total_capital
    
    return weights_dict

weights = capital_weights(mkt_value_of_debt, mkt_value_of_equity)

In [8]:
def weighted_average_cost_of_capital(cost_of_common, cost_of_debt, marginal_tax, weights_dict):
    """
    Summary: Calculate a firm's WACC.
    
    PARA cost_of_common: The firm's cost of common equity.
    PARA type: float
    
    PARA cost_of_debt: The firm's cost of debt.
    PARA type: float
    
    PARA marginal_tax: The firm's tax rate on an additional dollar of income.
    PARA type: float
    
    PARA weights_dict: The capital weights for each capital structure.
    PARA type: dictionary
    
    """    
    
    weight_debt = weights_dict['total_debt']
    weight_common = weights_dict['common_stock']
    tax = 1 - marginal_tax
    adj_cost_debt = cost_of_debt * tax
    
    return (weight_common * cost_of_common) + (adj_cost_debt * weight_debt)

# ROBLOX EXAMPLE
cost_equity = 0.064
marginal_tax = 0.27 # The tax rate for the last dollar of income

wacc = weighted_average_cost_of_capital(cost_equity, cost_of_debt, marginal_tax, weights)
print("Weighted Cost of Capital for %s is %.04f" % (company, wacc))

Weighted Cost of Capital for ROBLOX is 0.0651
